# **Signa_Lab ITESO:** Generador de *Embbeddings*

## **Cuaderno 02:** Generación de *embeddings*, reducción de dimensionalidades y clusterización.

Cuaderno de código para generar *embeddings* (relaciones semánticas codificadas en vectores) a partir de datos textuales, idealmente, procesados y depurados con antelación ([ver cuaderno 01](https://github.com/signalab/generador-embeddings/blob/main/cuadernos/01_Signa_Lab_generador_embeddings_Depuraci%C3%B3n_importar_limpiar_depurar_texto_01.ipynb)), con ayuda de modelos de lenguaje de la librería [sentence-transformers](https://www.sbert.net/), alojados en repositorios de [HuggingFace](https://huggingface.co/sentence-transformers) (en la nube) o descargados localmente.

**\***Los grupos de celdas marcadas con **asterisco requieren información** antes de seguir adelante.

## 1. Importar librerías, archivos de datos y modelos de lenguaje para la generación de *embeddings*


### Instalar e importar librerías:

**Instalar librerías:**

In [ ]:
# Instalar librerías de Python necesarias
!pip install nbformat
!pip install tf-keras
!pip install tensorflow_text
!pip install pandas
!pip install torch
!pip install sentence_transformers
!pip install numpy
!pip install operator
!pip install tqdm
!pip install transformers
!pip install tensorflow
!pip install nltk
!pip install matplotlib
!pip install scikit-learn
!pip install plotly
!pip install umap-learn
!pip install yellowbrick
!pip install ipywidgets

ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 615.3/615.3 MB 148.2 MB/s eta 0:00:01

**Importar librerías:**

In [ ]:
# Importar librerías de Python necesarias

import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm
import time

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import plotly.express as px
from sklearn.decomposition import PCA
import umap
from sklearn.manifold import TSNE
import nltk

from sklearn import datasets
from yellowbrick.cluster import SilhouetteVisualizer
import ipywidgets as widgets

from IPython.display import display, clear_output
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# Definir función para implementar barra de progreso sobre el procesamiento
def with_progress_bar(func):
    def wrapper(*args, **kwargs):
        # Inicia el contador de tiempo
        start_time = time.time()

        # Crea una barra de progreso
        pbar = tqdm(total=100, desc="Processing", ncols=70)

        # Define una función interna para actualizar la barra de progreso
        def update_progress_bar(i):
            pbar.update(i)

        # Agrega la función de actualización al diccionario de argumentos
        kwargs['update_progress_bar'] = update_progress_bar

        # Ejecuta la función original
        result = func(*args, **kwargs)

        # Finaliza la barra de progreso
        pbar.close()

        # Calcula y muestra el tiempo de ejecución
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Tiempo de ejecución: {execution_time:.2f} segundos")

        return result
    return wrapper

### *Indicar rutas de archivos de datos a importar y nombre de proyecto:

In [ ]:
# Definir función para cargar archivos a partir de la extensión en su ruta indicada
def load_file(path, encoding='utf-8'):
    if path.endswith('.csv'):
        return pd.read_csv(path, encoding=encoding)
    elif path.endswith('.xlsx'):
        return pd.read_excel(path)
    else:
        raise ValueError("Formato no compatible. Por favor carga solo archivos .csv or .xlsx.")

# Inicializar lista para alojar todas las rutas y una variable para el DataFrame final, accesible globalmente
file_paths = []
dfs = []
df = None  # DataFrame global

# Lista de codificaciones comunes
encoding_options = ['utf-8', 'latin1', 'ISO-8859-1']

# Dropdown para seleccionar la codificación del archivo
encoding_dropdown = widgets.Dropdown(
    options=encoding_options,
    value='utf-8',
    description='Codificación:',
    disabled=False,
)

# Campo de texto (input) para indicar nombre del proyecto (para integrarse en nombres de archivos a exportar)
project_name = widgets.Text(value='', placeholder='Escribe el nombre del proyecto (corto y sin espacios)', description='Nombre del proyecto:')

# Botones para añadir y eliminar archivos
add_button = widgets.Button(description="Añadir archivo", button_style='')
remove_button = widgets.Button(description="Eliminar archivo", button_style='warning')
load_button = widgets.Button(description="Cargar archivos", button_style='primary')

# Lista para almacenar los widgets de rutas de archivos
file_paths_widgets = []

# Función para añadir un campo de texto para la ruta del archivo
def add_file_input(b=None):
    file_path = widgets.Text(value='', placeholder='Escribe la ruta del archivo', description='Ruta del archivo:')
    file_paths_widgets.append(file_path)
    update_ui()

# Función para eliminar el último campo de texto de la ruta del archivo
def remove_file_input(b=None):
    if file_paths_widgets:
        file_paths_widgets.pop()
    update_ui()

# Función para procesar los archivos
def process_files(b=None):
    global df
    dfs.clear()
    for file_path in file_paths_widgets:
        path = file_path.value
        try:
            temp_df = load_file(path, encoding=encoding_dropdown.value)
            dfs.append(temp_df)
            print(f"Nombre de archivo: {path}")
            print(f"Filas/Columnas (shape): {temp_df.shape}")
        except ValueError as e:
            print(f"Error al cargar el archivo {path}: {e}")
            return

    if dfs:
        df = pd.concat(dfs, ignore_index=True)  # Concatenate all DataFrames
        print("\n¡Se cargaron todos los archivos!")
        print(f"Filas/Columnas (shape) de DataFrame creado: {df.shape}")

# Output widget para mostrar mensajes
output = widgets.Output()

# Función para actualizar las opciones del dropdown de columnas
def update_column_options():
    if df is not None:
        column_dropdown.options = df.columns.tolist()

# Definir función para actualizar UI
def update_ui():
    clear_output()
    display(project_name)
    display(encoding_dropdown)
    for path_input in file_paths_widgets:
        display(path_input)
    display(widgets.HBox([add_button, remove_button]))
    display(load_button)
    display(output)

# Inicializar UI con un campo de texto (input) para ruta de archivo
add_file_input()

# Conectar botones a funciones
add_button.on_click(add_file_input)
remove_button.on_click(remove_file_input)
load_button.on_click(process_files)

# Mostrar la UI inicial
update_ui()

### Previsualizar datos importados:

In [ ]:
# Imprimir columnas e información sobre datos importados
df.info

In [ ]:
# Previsualizar dataframe con CSVs importados
df

### *Indicar e importar modelo de lenguaje:

**Importar modelo de lenguaje para búsqueda semántica:**

\* Para ejecutar consultas de búsqueda semántica, se debe cargar el mismo modelo de lenguaje que el utilizado para generar *embeddings* en los datos importados. Por default, se cargará el modelo de software libre [intfloat/multilingual-e5-large-instruct](https://huggingface.co/intfloat/multilingual-e5-large-instruct) (Wang et al, 2024), con la librería de aprendizaje profundo [sentence-transformers](https://www.sbert.net/) (Reimers & Gurevych, 2019), pero pueden utilizarse otros del [repositorio en Hugging Face](https://huggingface.co/sentence-transformers) de dicha librería o cargados localmente.

**Cargar modelo de lenguaje** para su implementación en librería de *sentence-transformers*:

In [ ]:
embedder = None

pre_tested_models = [
    "intfloat/multilingual-e5-large-instruct",
    "all-mpnet-base-v2",
    "all-MiniLM-L12-v2",
    "paraphrase-multilingual-mpnet-base-v2",
    "facebook-dpr-ctx_encoder-multiset-base"
]

model_dropdown = widgets.Dropdown(
    options=pre_tested_models + ["other or local"],
    value=pre_tested_models[0],
    description='Model:',
    disabled=False,
)

custom_model_input = widgets.Text(
    value='',
    placeholder='Enter custom model path',
    description='Custom Path:',
    disabled=True
)

confirm_button = widgets.Button(
    description='Aceptar',
    disabled=False,
    button_style='primary'
)

output = widgets.Output()

def on_model_change(change):
    if change['new'] == "other or local":
        custom_model_input.disabled = False
    else:
        custom_model_input.disabled = True

model_dropdown.observe(on_model_change, names='value')

def load_model(b):
    global embedder
    with output:
        output.clear_output()
        model_path = model_dropdown.value
        if model_path == "other or local":
            model_path = custom_model_input.value
        embedder = SentenceTransformer(model_path)
        print(f"Model loaded from: {model_path}")

confirm_button.on_click(load_model)

display(model_dropdown, custom_model_input, confirm_button, output)

In [ ]:
print(embedder)

## 2. Generar *embeddings* con el modelo de lenguaje cargado

### \*Elegir columna de texto para generar *embeddings* en datos cargados:

**Definir función para generar embeddings:**

In [ ]:
# Definir función para generar embeddings
@with_progress_bar
def generate(mod, dfl, colText, update_progress_bar=None):
    tot = len(dfl)
    dfW = dfl
    dfW.loc[:, "Embedding"] = None
    for index, row in dfW.iterrows():
        # Iterar sobre el DataFrame y codificar cada texto
        embedding = mod.encode(str(row[colText])).tolist()
        dfW.at[index, 'Embedding'] = embedding
        # Actualizar la barra de progreso una vez por cada 10 filas
        if update_progress_bar is not None:
            update_progress_bar((index/tot)*2)
    return dfW

**Para un solo archivo**, generar embeddings con archivo de datos cargado en data frame:

In [ ]:
# Dropdown para seleccionar la columna del DataFrame
column_dropdown = widgets.Dropdown(
    options=df.columns.tolist(),
    description='Columna:',
    disabled=False,
)

# Botón para confirmar la selección y generar los embeddings
generate_button = widgets.Button(
    description='Generar Embeddings',
    disabled=False,
    button_style='primary'
)

# Output widget para mostrar mensajes
output = widgets.Output()

# Función para generar los embeddings
def generate_embeddings(b):
    global datasetEmbeddings
    with output:
        output.clear_output()
        textCol = column_dropdown.value
        # Asegúrate de que embedder esté definido globalmente
        datasetEmbeddings = generate(embedder, df, textCol)
        print(f"Embeddings generados para la columna: {textCol}")

generate_button.on_click(generate_embeddings)

# Mostrar widgets
display(column_dropdown, generate_button, output)

### Previsualizar *embeddings* generados:

**Previsualizar tabla con embeddings:**

In [ ]:
datasetEmbeddings.head()

### \*Exportar archivos de datos con *embeddings* generados (en formato CSV):

In [ ]:
# Input de texto para el nombre del archivo
file_name_input = widgets.Text(
    value='',
    placeholder='Escribe el nombre del archivo',
    description='Nombre Archivo:',
    disabled=False
)

# Botón para exportar el archivo
export_button = widgets.Button(
    description='Exportar CSV',
    disabled=False,
    button_style='success'
)

# Output widget para mostrar mensajes
output = widgets.Output()

# Función para exportar el archivo CSV
def export_csv(b):
    with output:
        output.clear_output()
        nombreArchivo = file_name_input.value
        datasetEmbeddings.to_csv(f"{nombreArchivo}Embeddings.csv")
        print(f"{nombreArchivo}Embeddings descargado en csv")

export_button.on_click(export_csv)

# Mostrar widgets
display(file_name_input, export_button, output)

## **3. Reducción de dimensiones con distintas técnicas (UMAP, PCA, TSNE)**

### *Elegir método para reducción de dimensionalidades:

**Elegir data frame** con embeddings para reducir dimensionalidaes y clusterizar:

In [ ]:
# Elegir data frame con embeddings (df, en caso de un solo archivo, df1,2,3... e caso de múltiples archivos:

# Un solo archivo cargado con embeddings generados
dfClusters = datasetEmbeddings

Definir función para reducir dimensionalidades con: **UMAP**

In [ ]:
# Definir función para UMAP
def genUMAP(df, columnaEmbeddings, nDims):
    dfW = df
    umap_model = umap.UMAP(n_components=nDims)
    X_umap = umap_model.fit_transform(dfW[columnaEmbeddings].tolist())
    dfW["embeddingsReducidos"] = X_umap.tolist()
    return dfW

Definir función para reducir dimensionalidades con: **PCA**

In [ ]:
# Definir función para PCA
def genPCA(df, columnaEmbeddings, nDims):
    dfW = df
    pca = PCA(n_components=nDims)
    X_pca = pca.fit_transform(dfW[columnaEmbeddings].tolist())
    dfW["embeddingsReducidos"] = list(X_pca)
    return dfW

Definir función para reducir dimensionalidades con: **TSNE**

In [ ]:
# Definir función para TSNE
def genTSNE(df, columnaEmbeddings, nDims):
    dfW = df
    tsne = TSNE(n_components=nDims, random_state=42)
    X_tsne = tsne.fit_transform(dfW[columnaEmbeddings].tolist())
    dfW["embeddingsReducidos"] = list(X_tsne)
    return dfW

**Definir función para elegir método para reducir dimensiones:**

In [ ]:
def reduceDim(df, columnaEmbeddings, nDims, method):
    if method.lower() == 'umap':
        return genUMAP(df, columnaEmbeddings, nDims)
    elif method.lower() == 'pca':
        return genPCA(df, columnaEmbeddings, nDims)
    elif method.lower() == 'tsne':
        return genTSNE(df, columnaEmbeddings, nDims)

**Elegir método a utilizar, número de dimensiones (2D o 3D) y ejecutar reducción de dimensionalidades sobre *embeddings* cargados:**

In [ ]:
# Indicar nombre de columna con embeddings completos (vector dado por las dimensiones del modelo de lenguaje)
columnaConEmbeddings = "Embedding"

# Indicar si se busca reducir a 2 o 3 dimensiones para visualizador 2D o 3D
numDimensiones = 3

# Dropdown para elegir el método de reducción de dimensionalidades
metodo_dropdown = widgets.Dropdown(
    options=["umap", "pca", "tsne"],
    value="umap",
    description='Método:',
    disabled=False,
)

# Botón para confirmar la selección y ejecutar la reducción de dimensionalidades
reduce_button = widgets.Button(
    description='Reducir Dimensiones',
    disabled=False,
    button_style='primary'
)

# Output widget para mostrar mensajes
output = widgets.Output()

# Función para ejecutar la reducción de dimensionalidades
def reducir_dimensiones(b):
    global datasetConDimensionesReducidas
    with output:
        output.clear_output()
        metodo = metodo_dropdown.value
        datasetConDimensionesReducidas = reduceDim(dfClusters, columnaConEmbeddings, numDimensiones, metodo)
        print(f"Reducción de dimensionalidades completada usando el método: {metodo}")

reduce_button.on_click(reducir_dimensiones)

# Mostrar widgets
display(metodo_dropdown, reduce_button, output)

### Previsualizar tabla con *embeddings* reducidos a 3 dimensiones:

In [ ]:
datasetConDimensionesReducidas.head()

## **4. Clusterización y visualización de *embeddings***

### Visualizar gráficas para identificar número óptimo de clústeres:

Definir función para **Método del Codo** para calcular número ideal de clústers para los datos cargados:

In [ ]:
def elbowMethod(df, columnWiEmbeddings, rangoPosibleDeCluster, byRegion=False):
    # Convierte la columna de embeddings del DataFrame a un array de numpy para su procesamiento
    embeddings = np.array(df[columnWiEmbeddings].tolist())
    # Calcula la inercia para diferentes valores de k (número de clústeres)
    # Inicializa una lista vacía para almacenar los valores de inercia calculados para cada número de clústeres
    inertia_values = []
    # Define un rango de valores de k (número de clústeres) para probar, basándose en el rango proporcionado como argumento
    possible_k_values = range(rangoPosibleDeCluster[0], rangoPosibleDeCluster[1]+1)  # Prueba k desde n1 hasta n2 clústeres

    # Itera sobre el rango de posibles valores de k
    for index, k in enumerate(possible_k_values):
        # Inicializa el algoritmo KMeans con el número actual de clústeres (k) y una semilla aleatoria fija
        kmeans = KMeans(n_clusters=k, random_state=0)

        # Entrena el modelo KMeans con los embeddings
        kmeans.fit(embeddings)

        # Añade la inercia del modelo (la suma de las distancias cuadradas de las muestras a su centro de clúster más cercano) a la lista de valores de inercia
        inertia_values.append(kmeans.inertia_)

    # Prepara los resultados para ser graficados, conteniendo los valores de k y las inercias correspondientes
    resToPlot = [possible_k_values, inertia_values]

    # Grafica el método del codo utilizando matplotlib, donde el eje X representa el número de clústeres y el eje Y la inercia
    plt.plot(resToPlot[0], resToPlot[1], marker='o')
    plt.xlabel('Número de Clústeres (k)')
    plt.ylabel('Inercia')
    plt.title('Método del Codo para Determinar k')
    plt.show()

**Ejecutar método del codo** para identificar número deseable de clústers:

In [ ]:
# Indicar nombre de columna con embeddings reducidos
columnaConEmbeddings = "embeddingsReducidos"
# Calcular número ideal en un rango de 1 a 20 clústers
rangoDeClusters = [1,20]
elbowMethod(datasetConDimensionesReducidas, columnaConEmbeddings, rangoDeClusters)

Definir función para **Método de Silueta** para calcular número ideal de clústers para los datos cargados

In [ ]:
def silhouetteMethod(df, columnWithEmbeddings, rangeOfClusters):
    # Convierte la columna de embeddings del DataFrame a un array de numpy para su procesamiento
    embeddings = np.array(df[columnWithEmbeddings].tolist())

    # Establece el número de subplots basado en el rango de clústeres a analizar
    n_clusters = range(rangeOfClusters[0], rangeOfClusters[1] + 1)
    n_rows = len(n_clusters) // 2 + len(n_clusters) % 2
    fig, ax = plt.subplots(n_rows, 2, figsize=(16, 32))
    plt.subplots_adjust(hspace=0.5)

    # Itera sobre el rango especificado de número de clústeres
    for i, k in enumerate(n_clusters):
        # Crea una instancia de KMeans para el número actual de clústeres
        km = KMeans(n_clusters=k, init='k-means++', n_init=10, max_iter=300, random_state=42)

        # Calcula la fila y columna para el subplot actual
        q, mod = divmod(i, 2)

        # Crea una instancia de SilhouetteVisualizer para visualizar el coeficiente de silueta
        visualizer = SilhouetteVisualizer(km, colors='yellowbrick', ax=ax[q][mod])

        # Ajusta el visualizador con los embeddings
        visualizer.fit(embeddings)

        # Establece el título del subplot
        ax[q][mod].set_title(f'Clusters: {k}')

    # Ajusta la disposición de los subplots para evitar solapamientos
    plt.tight_layout()
    plt.show()

In [ ]:
# Ejecutar función de método de silueta
silhouetteMethod(dfClusters, 'embeddingsReducidos', (2, 20))

Definir función de clusterizarización utilizando método **K-Means:**

In [ ]:
# Definir función de clusterización con K-means
def clusterKmeans(df, optimal_k_value, columnaEmbeddings, byRegion=False, cluster_col_name=None):
    embeddings = df[columnaEmbeddings].tolist()
    dfW = df.copy() # Crear copia de trabajo de DataFrame

    if cluster_col_name is None:
      cluster_col_name = "cluster"

    try:
        if byRegion: # Clusterización por región (opcional, no utilizado en metodología final)
            # Inicializar nueva columna para alojar etiquetas de clústeres
            dfW[cluster_col_name] = None

            # Iterar sobre cada región (opcional)
            for region, df_region in dfW.groupby('region'):
                # Aplicar clusterización de K-means en cada región (opcional)
                kmeans = KMeans(n_clusters=optimal_k_value, random_state=0)
                cluster_labels = kmeans.fit_predict(df_region[columnaEmbeddings].tolist())

                # Añadir prefijo de clúster por región (opcional)
                prefixed_cluster_labels = [f"{region}_{label}" for label in cluster_labels]

                # Asignar etiquetas de clúster por región a DataFrame
                dfW.loc[dfW['region'] == region, cluster_col_name] = prefixed_cluster_labels
        else:
            # Aplicar clusterización por K-means a todo el conjunto de datos importado (versión implementada en metodología final)
            kmeans = KMeans(n_clusters=optimal_k_value, random_state=0)
            cluster_labels = kmeans.fit_predict(embeddings)
            dfW[cluster_col_name] = cluster_labels

        return dfW
    except Exception as e:
        print(f"Error al generar clusters: {e}")
        return pd.DataFrame({0: ["Error al generar clusters"]})


### *Definir número de clústeres a segmentar:

**Ejecutar clusterización** con número deseado de clúster con método *k-means:*

In [ ]:
# Indicar nombre de columna con embeddings reducidos
columnaConEmbeddings = "embeddingsReducidos"

# Input para el número de clusters
numero_clusters_input = widgets.IntText(
    value=7,
    description='Núm. Clusters:',
    disabled=False
)

# Botón para confirmar la selección y ejecutar la segmentación en clusters
cluster_button = widgets.Button(
    description='Segmentar Clusters',
    disabled=False,
    button_style='success'
)

# Output widget para mostrar mensajes
output = widgets.Output()

# Función para ejecutar la segmentación en clusters
def segmentar_clusters(b):
    global dfClusterizado
    with output:
        output.clear_output()
        numeroDeClusters = numero_clusters_input.value
        dfClusterizado = clusterKmeans(datasetConDimensionesReducidas, numeroDeClusters, columnaConEmbeddings, byRegion=False)
        print(f"Segmentación en {numeroDeClusters} clusters completada.")

cluster_button.on_click(segmentar_clusters)

# Mostrar widgets
display(numero_clusters_input, cluster_button, output)

### Previsualizar tabla con clústeres segmentados:

In [ ]:
# Previsualizar tabla de registros con sus respectivos clústers asignados
dfClusterizado

### Visualizar en 3D *embeddings* generados y clústeres:

Definir función para **visualizar relaciones semánticas y clústeres en 3D:**

In [ ]:
def visualize3D(df, columnaEmbeddingsReducidas, columnaTexto, byCategory=False, columnaCluster=None):
  if columnaCluster is None:
    columnaCluster = "cluster"
  embeddingWithSelectedDimensions = df[columnaEmbeddingsReducidas].tolist()
  dfToPlot = {
        "X": [x[0] for x in embeddingWithSelectedDimensions],
        "Y": [y[1] for y in embeddingWithSelectedDimensions],
        "Z": [z[2] for z in embeddingWithSelectedDimensions],
        "cluster": df[columnaCluster].tolist(),
        "text": df[columnaTexto]
  }

  fig = px.scatter_3d(dfToPlot, x='X', y='Y', z='Z', color='cluster', hover_data=["text"],
                         labels={'X': 'Dimensión 1', 'Y': 'Dimensión 2', 'Z': 'Dimensión 3'},
                         title=f'Visualización de clústers semánticos',
                         color_discrete_sequence=px.colors.sequential.Viridis)
  fig.show()

**Ejecutar visualización de clústers 3D:**

In [ ]:
# Indicar nombre de columna con embeddings reducidos
columnaConEmbeddings = "embeddingsReducidos"
# Indicar columna de texto para etiquetas
columnaTextoClusterizado = "clean_text"
visualize3D(dfClusterizado, columnaConEmbeddings, columnaTextoClusterizado, columnaCluster="cluster")

### \*Exportar archivos de datos con *embeddings* generados y clústeres:

**Exportar archivos de datos (formato CSV y JSON) con clústers calculados:**

In [ ]:
# Dropdown para seleccionar el formato de exportación
format_dropdown = widgets.Dropdown(
    options=['csv', 'json'],
    value='csv',
    description='Formato:',
    disabled=False,
)

# Input de texto para el nombre del archivo
file_name_input = widgets.Text(
    value='',
    placeholder='Escribe el nombre del archivo',
    description='Nombre Archivo:',
    disabled=False
)

# Botón para exportar el archivo
export_button = widgets.Button(
    description='Exportar',
    disabled=False,
    button_style='success'
)

# Output widget para mostrar mensajes
output = widgets.Output()

# Función para exportar el archivo
def export_file(b):
    with output:
        output.clear_output()
        file_name = file_name_input.value
        file_format = format_dropdown.value
        if file_format == 'csv':
            dfClusterizado.to_csv(f"{file_name}.csv", index=False)
            print(f"Archivo {file_name}.csv exportado exitosamente.")
        elif file_format == 'json':
            dfClusterizado.to_json(f"{file_name}.json", orient='records', lines=True)
            print(f"Archivo {file_name}.json exportado exitosamente.")

export_button.on_click(export_file)

# Mostrar widgets
display(format_dropdown, file_name_input, export_button, output)

### Postprocesar datos para visualización en Cosmograph:

Preparar dataset clusterizado para su visualización en **Cosmograph**:

- Crear columna **cluster-str** basada en cluster. Cambiar valores a letras en orden alfabético.
- Crear nueva columna **x**  a partir del primer valor de reduced_embeddings.
- Crear nueva columna **y**  a partir del segundo valor de reduced_embeddings.

In [ ]:
# Definir la función para postprocesar los clusters y agregar nuevas columnas
def postprocessClusters(dfClusterizado):
    # Crear un diccionario para convertir números a letras (A-Z)
    number_to_letter = {
        0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E',
        5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
        10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O',
        15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T'
    }

    # Crear las nuevas columnas
    dfClusterizado['cluster-str'] = dfClusterizado['cluster'].map(number_to_letter)
    dfClusterizado['x'] = dfClusterizado['embeddingsReducidos'].apply(lambda value: value[0])
    dfClusterizado['y'] = dfClusterizado['embeddingsReducidos'].apply(lambda value: value[1])

    # Devolver el nuevo dataframe con las columnas agregadas
    global dfFinal
    dfFinal = dfClusterizado.copy()

# Llamar a la función para procesar el dataframe
postprocessClusters(dfClusterizado)

# Mostrar las primeras filas del nuevo dataframe
dfFinal.head()


**Exportar archivo de datos (formato CSV) procesado para su compatibilidad con la aplicación [Cosmograph](https://cosmograph.app):**

In [ ]:
dfFinal.to_csv(f"{project_name.value}_cosmograph.csv", index=False)

## **5. Referencias:**

* Bird, Steven, Edward Loper & Ewan Klein (2009). Natural Language Processing with Python.  O'Reilly Media Inc.
* McInnes, L., Healy, J., & Melville, J. (2018). Umap: Uniform manifold approximation and projection for dimension reduction. arXiv preprint arXiv:1802.03426. https://arxiv.org/abs/1802.03426
* Pedregosa, F., Varoquaux, G., Gramfort, A., Michel, V., Thirion, B., Grisel, O., Blondel, M., Prettenhofer, P., Weiss, R., Dubourg, V., Vanderplas, J., Passos, A., Cournapeau, D., Brucher, M., Perrot, M., Duchesnay, E., & others. (2011). Scikit-learn: Machine Learning in Python. Journal of Machine Learning Research, 12, 2825–2830.
* Reimers, N., & Gurevych, I. (2019). Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks (arXiv:1908.10084). arXiv. http://arxiv.org/abs/1908.10084
* Rousseeuw, P. (1987). Silhouettes: a Graphical Aid to the Interpretation and Validation of Cluster Analysis. Computational and Applied Mathematics. 20: 53–65. doi:10.1016/0377-0427(87)90125-7.
* Spärck-Jones, K. (1972). A statistical interpretation of term specificity and its application in retrieval. Journal of Documentation, 28(1), 11-21. https://www.staff.city.ac.uk/~sbrp622/idfpapers/ksj_orig.pdf
* Thorndike, R. (1953). Who Belongs in the Family?. Psychometrika. 18 (4): 267–276. doi:10.1007/BF02289263. S2CID 120467216.
* Wang, L., Yang, N., Huang, X., Yang, L., Majumder, R., & Wei, F. (2024). Multilingual E5 Text Embeddings: A Technical Report. arXiv preprint arXiv:2402.05672. Recuperado de https://arxiv.org/abs/2402.05672

*Programación asistida ocasionalmente con herramientas de IA Generativa: ChatGPT, Phind, Google Gemini y Perplexity


## **6. Créditos**

**Realizado por el equipo de Signa_Lab ITESO:**

- **Programación de cuadernos de código (Python)**:
Javier de la Torre Silva, José Luis Almendarez González y Diego Arredondo Ortiz

- **Supervisión del desarrollo tecnológico y documentación:**
Diego Arredondo Ortiz

- **Equipo de Coordinación Signa_Lab ITESO:**
Paloma López Portillo Vázquez, Víctor Hugo Ábrego Molina y Eduardo G. de Quevedo Sánchez

Mayo, 2024. Instituto Tecnológico y de Estudios Superiores de Occidente (ITESO)
Tlaquepaque, Jalisco, México.
